In [1]:
import wandb
from youtube import download_youtube_audio

In [2]:
url = "https://www.youtube.com/watch?v=-kFOXP026eE&list=PLS3_1JNX8dEh5YcO-Y05stU0u_T9nqIlF&index=1"

In [3]:
audio_file = download_youtube_audio(url)
audio_file

'/home/tcapelle/wandb/whisper/./richard_feynman_messenger_lectures_at_cornell__the_character_of_physical_law__part_1_the_law_of_gravitation.mp4'

In [4]:
audio_file

'/home/tcapelle/wandb/whisper/./richard_feynman_messenger_lectures_at_cornell__the_character_of_physical_law__part_1_the_law_of_gravitation.mp4'

In [19]:
with wandb.init(project="whisper", job_type="log_artifact"):
    at = wandb.Artifact("audio_file", type="audio", description="Richard Feymann Lectures at Cornell: Part 1", 
                        metadata={"url":url})
    at.add_file(audio_file)
    wandb.log_artifact(at)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin


## Whisper

In [1]:
import torch
import wandb
import whisper
from whisper.transcribe import write_vtt
from types import SimpleNamespace

from pathlib import Path

In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "tiny.en"
artifact = 'capecape/whisper/audio_file:v0'
url = "https://www.youtube.com/watch?v=-kFOXP026eE&list=PLS3_1JNX8dEh5YcO-Y05stU0u_T9nqIlF&index=1"
fname = "richard_feynman_messenger_lectures_at_cornell__the_character_of_physical_law__part_1_the_law_of_gravitation.mp4"

In [33]:
def transcribe_file(model, file, log=True, language="en", log_predictions=False):
    "Run whisper on an audio file and save results"
    print(f"Transcribing file: {file}")
    
    result = model.transcribe(str(file), verbose=False, log=log, language=language)
    out_file = Path.cwd()/Path(file).name
    
    # save TXT
    with open(out_file.with_suffix(".txt"), "w", encoding="utf-8") as txt:
        print(result["text"], file=txt)
    if log_predictions:
        at = wandb.Artifact("model_predictions", type="text", description="Models output for Feynman's lecture")
        at.add_file(out_file.with_suffix(".txt"))
        wandb.log_artifact(at)
        wandb.run.log_code(".", include_fn=lambda path: path.endswith(".txt") )
    # save VTT
    with open(out_file.with_suffix(".vtt"), "w", encoding="utf-8") as vtt:
        write_vtt(result["segments"], file=vtt)
    return result

In [34]:
models = "tiny.en,base.en,small.en,medium.en".split(",")

In [ ]:
for _ in range(1):
    for model_name in models:
        model = whisper.load_model(model_name).to(device)
        config = SimpleNamespace(model=model_name, device=device, url=url, fname=fname, artifact=artifact)    
        with wandb.init(project="whisper", job_type="bench", config=config):
            artifact = wandb.use_artifact('capecape/whisper/audio_file:v0', type='audio')
            audio_file = Path(artifact.download())/fname
            res = transcribe_file(model, audio_file, log_predictions=True)